In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-annotated/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'label','commit_message':'message','commit_diff':'diff'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_2316/1556623075.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,label,diff,message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_DEL_LINE,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE
commit_sha,,,,,,,,,,,,,,,,,,,,,
be937a3290223f926fe50684f1344569a573ed4b,1,diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
c800d2e36954edddcb83aa1df7f623f2780c7780,0,diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
469e56eeff17eb857c95e935ca7b49723c43470e,2,diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...,0,0,0,0,0,0,0,...,0,0,8,0,0,15,0,0,0,0
24f5d302c0906f3377f4d6bcc2735d943ce13e3d,0,diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,...",3,37,0,0,9,0,0,...,0,0,0,0,0,0,1,2,0,0
1effdb7b959503596b07f7f3e74618ab63f9e5e5,2,diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...,1,0,12,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,1,diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,1,diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...,2,6,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,1,diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...,1,1,5,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/eval.csv',index=False)

In [5]:
train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=50, random_state=42))

In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train

,label,diff,message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_DEL_LINE,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE
commit_sha,,,,,,,,,,,,,,,,,,,,,
e0b52079daeec08e5d349d686e217c8ba11c1f64,0,diff --git a/crates/bevy_render/src/renderer/r...,Implement RenderResource for Box<T> (#1893)\n\...,1,17,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ddff0f642acbf650b53d1ddffe544900569ab983,2,diff --git a/examples/js/SimulationRenderer.js...,Add comments to FF RGB format fix\n\n,2,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57f2d7de58427a413c406ef0a5a4e120e3351620,2,diff --git a/docs/00-Getting-Started.md b/docs...,Update docs and add plugin section (suggested ...,0,0,0,0,0,0,0,...,0,0,3,8,0,3,0,0,0,0
b00e9fc227dc2ab1070326f29fbe9626702dfa67,1,diff --git a/helix-term/src/commands.rs b/heli...,Handle line endings correctly in surround\n\n,2,5,0,2,0,8,0,...,0,0,0,0,0,0,0,0,0,0
c168a92f187943a401f6c4095b5abde44478d0c9,2,diff --git a/components/tag/demo/control.md b/...,docs: add missing key prop for controlled tag ...,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
defd45ccbadb7e0e347b1999d48fe354998ba37a,2,diff --git a/src/compiler/optimizer.js b/src/c...,Typo fix: optimizier -> optimizer (#4015)\n\n\n,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
bd34203bc40d6a406dfac3ab6cb41f5dfed63a77,1,diff --git a/packages/vite/src/node/plugins/im...,fix: Avoid importing in source that __vitePrel...,1,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c3842edd8ea5da4626ffe6cb7f6c1965fa956757,2,diff --git a/keras/engine/training.py b/keras/...,Corrected 3 small typos in training.py error m...,1,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)


base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 0.7626: 100%|██████████| 139/139 [01:30<00:00,  1.54batch/s]


=============================train========================


Epoch 2/10 Loss: 0.3620: 100%|██████████| 139/139 [01:25<00:00,  1.63batch/s]


=============================train========================


Epoch 3/10 Loss: 0.1672: 100%|██████████| 139/139 [01:24<00:00,  1.64batch/s]


...

=============================train========================


Epoch 9/10 Loss: 0.0022: 100%|██████████| 139/139 [01:25<00:00,  1.63batch/s]


=============================train========================


Epoch 10/10 Loss: 0.0274: 100%|██████████| 139/139 [01:25<00:00,  1.63batch/s]


=============================train========================
Validation Accuracy: 0.7511
Precision: 0.7838
Recall: 0.7511
F1-Score: 0.7282
              precision    recall  f1-score   support

           0     0.5676    0.6176    0.5915        34
           1     0.9565    0.3667    0.5301        60
           2     0.7627    0.9441    0.8438       143

    accuracy                         0.7511       237
   macro avg     0.7623    0.6428    0.6551       237
weighted avg     0.7838    0.7511    0.7282       237



In [9]:
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.7983
Precision: 0.8112
Recall: 0.7983
F1-Score: 0.7805
              precision    recall  f1-score   support

           0     0.6875    0.6875    0.6875        32
           1     0.9130    0.4118    0.5676        51
           2     0.8033    0.9484    0.8698       155

    accuracy                         0.7983       238
   macro avg     0.8013    0.6826    0.7083       238
weighted avg     0.8112    0.7983    0.7805       238



In [11]:
res = test.copy()

In [12]:
res['test_labels'] = test_labels
res['test_probabilities'] = test_probabilities
res['test_embeddings'] = test_embeddings
res['test_predictions'] = test_predictions

In [13]:
# PR curve
combined_model.plot_pr_curve(test_labels, test_probabilities)
# Plot t-SNE for the best model

ValueError: multiclass format is not supported

In [ ]:
# cluster to show the quality of embeddings
combined_model.plot_tsne(test_embeddings, test_labels)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix on Dataset I', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    # plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
cm = confusion_matrix(test_labels, test_predictions)

# Call the function to plot the confusion matrix
class_names = test['label'].unique()  # Replace with your class names
plot_confusion_matrix(cm, classes=class_names, normalize=True)  # Set normalize=False if you want counts instead of percentages
plt.savefig('Confusion Matrix on Dataset Patch_DB')

In [ ]:
res.to_csv('combined_model_I.csv')

In [ ]:
res_mis = test.copy()

In [ ]:
# test_data

In [ ]:
# 设置数据缺失的情况
# 设置随机种子以便结果可重复
import numpy as np
def data_missing(df):
    
    np.random.seed(42)
    
    # 定义缺失值的比例
    missing_rate = 0.2
    
    # 计算每列需要创建缺失值的数量
    num_missing = int(missing_rate * len(df))
    
    # 随机选择行进行缺失
    for col in ['message', 'diff']:
        missing_indices = np.random.choice(df.index, num_missing, replace=False)
        
        # 选择每个单元格的部分字符来替换，而不是全部替换成NaN
        for idx in missing_indices:
            if np.random.rand() > 0.5:
                # 保留前半部分
                try:
                    # print(df.loc[idx, col][:len(df.loc[idx, col])//2] + '...')
                    df.loc[idx, col] = df.loc[idx, col][:len(df.loc[idx, col])//2] + '...'
                except:
                    continue
            else:
                # 保留后半部分
                try:
                    # print('...' + df.loc[idx, col][len(df.loc[idx, col])//2:])
                    df.loc[idx, col] = '...' + df.loc[idx, col][len(df.loc[idx, col])//2:]
                except:
                    continue
    
    return df
test_data_miss = data_missing(test)

In [ ]:
test_data_miss

In [ ]:
test_data_miss['message']

In [ ]:
test_data_miss[test_data_miss['message'].str.contains('...')]

In [ ]:
# test_data_miss.reset_index(inplace=True)
test_miss_dataset = preprocesser.SentencePairDataset(test_data_miss, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_miss_loader = DataLoader(test_miss_dataset, batch_size=8, shuffle=False)
test_miss_loader

In [ ]:
test_miss_acc, test_miss_labels, test_miss_probabilities, test_miss_embeddings, test_miss_predictions = combined_model.evaluate(test_miss_loader)

In [ ]:
res_mis['test_miss_labels'] = test_miss_labels
res_mis['test_miss_probabilities'] = test_miss_probabilities
res_mis['test_miss_embeddings'] = test_miss_embeddings
res_mis['test_miss_predictions'] = test_miss_predictions

In [ ]:
res_mis.to_csv('combined_model_res_data_missing_I.csv')